In [ ]:
!pip install inference_sdk

In [ ]:
import os
import json
from inference_sdk import InferenceHTTPClient

# Initialize the client
CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="L5uh8sOgmpA0sglsoQzM"  # Replace this with your actual API key
)

# Define the model ID
model_id = "license-plate-recognition-rxg4e/6"

# Define the input and output directories
input_dir = "/content/dataset"
output_dir = "/content/cords"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Process each PNG file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".png"):
        image_path = os.path.join(input_dir, filename)
        try:
            # Perform inference
            result = CLIENT.infer(image_path, model_id=model_id)

            # Save the result as a JSON file
            output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.json")
            with open(output_path, "w") as json_file:
                json.dump(result, json_file, indent=4)

            print(f"Processed {filename} and saved results to {output_path}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")

In [ ]:
import cv2
import json
import os

# Define directories based on your structure
CORDS_DIR = "./cords"
DATASET_DIR = "./dataset"

def draw_bounding_boxes(json_file):
    # Load JSON data
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Extract the base name (file name without extension) to find the image
    base_name = os.path.splitext(os.path.basename(json_file))[0]
    image_extensions = ['.jpg', '.png', '.jpeg']  # Common image extensions
    image_file = None

    # Search for the image in the dataset directory
    for ext in image_extensions:
        potential_image = os.path.join(DATASET_DIR, base_name + ext)
        if os.path.exists(potential_image):
            image_file = potential_image
            break

    if not image_file:
        print(f"Image file not found for JSON: {json_file}")
        return

    # Read the image
    image = cv2.imread(image_file)
    if image is None:
        print(f"Unable to load image: {image_file}")
        return

    # Draw bounding boxes from predictions
    predictions = data.get("predictions", [])
    for pred in predictions:
        x, y, width, height = pred["x"], pred["y"], pred["width"], pred["height"]
        confidence = pred["confidence"]

        # Convert center coordinates to top-left corner coordinates
        x1 = int(x - width / 2)
        y1 = int(y - height / 2)
        x2 = int(x + width / 2)
        y2 = int(y + height / 2)

        # Draw the rectangle and add a label
        label = f"{pred['class']} ({confidence:.2f})"
        color = (0, 255, 0)  # Green for the bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Save the image with annotations in the dataset directory
    output_file = os.path.join(DATASET_DIR, base_name + "_annotated.jpg")
    cv2.imwrite(output_file, image)
    print(f"Annotated image saved as: {output_file}")

# Process all JSON files in the cords directory
def process_all_jsons():
    for file in os.listdir(CORDS_DIR):
        if file.endswith(".json"):
            json_path = os.path.join(CORDS_DIR, file)
            draw_bounding_boxes(json_path)

# Execute the script
process_all_jsons()